Neste exemplo ambos os autómatos têm o mesmo conjunto de eventos: $$\,\{\mathbf{init},\mathbf{off},\mathbf{on}\}\,$$.

In [3]:
from pysmt.shortcuts import *
from pysmt.typing import *

INIT = Int(-1)
OFF = Int(0)
ON = Int(1)

ns se a ordem vai ser esta

In [4]:
def traffic(s):
    s = {}
    s["green_A"] = Symbol("green_A", BOOL)  
    s["green_B"] = Symbol("green_B", BOOL)  
    s["yellow_A"] = Symbol("yellow_A", BOOL) 
    s["yellow_B"] = Symbol("yellow_B", BOOL)
    s["t"] = Symbol("t", REAL)               
    return s
def traffic_flow(s):
    return Equals(s["t_dot"], Real(1))

def semaforo_update(s):
    """
    green = avança
    yellow = opcional
    red = não avança
    """

    green_A = Not(Equals(s["sector_A"], s["sector_B"]))
    green_B = Not(Equals(s["sector_B"], s["sector_A"]))

    yellow_A = And(LT(s["sector_A"], s["sector_B"]), Not(Equals(s["sector_A"] + 1, s["sector_B"])))
    yellow_B = And(GT(s["sector_B"], s["sector_A"]), Not(Equals(s["sector_B"] - 1, s["sector_A"])))

    return And(
        Equals(s["green_A"], green_A),
        Equals(s["green_B"], green_B),
        Equals(s["yellow_A"], yellow_A),
        Equals(s["yellow_B"], yellow_B)
    )


def can_advance_A(s):
    return Or(s["green_A"], s["yellow_A"])


def can_advance_B(s):
    return Or(s["green_B"], s["yellow_B"])


Autómato do navio A→B


    - Os modos do navio  são definidos pelos sectores $$\,s\,$$. A cada modo estão associados  parâmetros racionais, constantes dentro de cada sector mas variando de sector para sector.
        - $$\gamma > 0\,$$  é a aceleração linear  definida pela propulsão do navio
        - $$\varepsilon\;$$   é a  força exercida pela corrente no canal que actua a favor ou contra a trajectória.
        - $$\phi\,$$ é o rumo do navio definido como o ângulo que a trajetória do movimento linear faz com o eixo horizontal.
        - $$V\,$$ é o limite superior na velocidade do navio. 

ZONA DE ACELERAÇÃO : $$\{s_{11},s_{13}\}\,$$ $$\{s_2,s_4\}\,$$  $$\,\gamma \simeq 1\,$$
ZONA DE DESACELERAÇÃO: $$\,\{s_{1},s_{3}\}\,$$  $$\,\{s_{12},s_{14}\}\,$$ $$\,\gamma \simeq 0\,$$
VELOCIDADE CONSTANTE(e de cruzeiro?): $$\{s_5,s_7,s_9\}\,$$  $$\{s_6,s_8,s_{10}\}\,$$
ZONA DE VELOCIDADE BAIXA +- CONSTANTE: $$\{s_0\}\,$$

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import REAL, INT
import math 
# (0,1) origem no canto inferior esquerdo
# ============================================================
# 1. VARIÁVEIS DO NAVIO A 
# ============================================================
def VAR_A_B(i):
    s = {}
    # estado discreto
    s["sector_A"] = Symbol("sector_A" + str(i), INT)

    # variáveis contínuas de estado
    s["z_A"]   = Symbol("z_A"+ str(i), REAL)
    s["v_A"]   = Symbol("v_A"+ str(i), REAL)
    s["tau_A"] = Symbol("tau_A"+ str(i), REAL)

    #sendo $$(x_0,y_0)\,$$ o ponto do sector onde a trajetória se inicia, no inicio inicio é (0,1)
    s["x0_A"] = Symbol("x0_A"+ str(i), REAL)
    s["y0_A"] = Symbol("y0_A"+ str(i), REAL)

    # parâmetros associados ao modo (sector)
    # Os modos do navio  são definidos pelos sectores $$\,s\,$$. 
    # A cada modo estão associados  parâmetros racionais, constantes dentro de cada sector mas variando de sector para sector.

    s["gamma_A"]   = Symbol("gamma_A"+ str(i), REAL) #  aceleração linear  definida pela propulsão do navio
    s["epsilon_A"] = Symbol("epsilon_A"+ str(i), REAL) #é a  força exercida pela corrente no canal que actua a favor ou contra a trajectória
    s["phi_A"]     = Symbol("phi_A"+ str(i), REAL) #  é o rumo do navio definido como o ângulo que a trajetória do movimento linear faz com o eixo horizontal.
    s["V_A"]       = Symbol("V_A"+ str(i), REAL) # limite superior na velocidade do navio

    # o coeficiente de atrito com a àgua, uma constante positiva que é independente do modo mas depende do navio.
    s["sigma_A"] = Symbol("sigma_A"+ str(i), REAL)

    s["v_A_dot"] = Symbol("v_A_dot", REAL)
    s["z_A_dot"] = Symbol("z_A_dot", REAL)
    return s
#---------------------------------------
#x_A = x0_A + z_A * cos(phi_A) #calculo q ta no exercicio
#y_A = y0_A + z_A * sin(phi_A) #calculo q ta no exercicio ver onde q vou colcoar isto
"""
VER ONDE COLOCAR
position_A = And(
    Equals(s["x_A"], Plus(s["x0_A"], Times(s["z_A"], Cos(s["phi_A"])))),
    Equals(s["y_A"], Plus(s["y0_A"], Times(s["z_A"], Sin(s["phi_A"]))))
)
Equals(s["x_B"], Plus(s["x0_B"], Times(s["z_B"], Cos(s["phi_B"])))),
Equals(s["y_B"], Plus(s["y0_B"], Times(s["z_B"], Sin(s["phi_B"]))))

"""



# podemos ver q x0_A é 0 e y0_A é 1
# nao sei se e para colocar isto
def init_A_B(s):
    return And(
        Equals(s["sector_A"], Int(11)), # tou a supor q ele comeca ou no setor 11 ou no 13
        Equals(s["z_A"], Real(0)),
        GE(s["v_A"], Real(0)),
        Equals(s["tau_A"],Real(0)),
        Equals(s["x0_A"], Real(0)),
        Equals(s["y0_A"], Real(1)),
    )


# os modos de aceleracao e assim
def acceleration_zone_A_B(s):
    return Or(
        Equals(s["sector_A"], Int(11)),
        Equals(s["sector_A"], Int(13)),
        Equals(s["sector_A"], Int(2)),
        Equals(s["sector_A"], Int(4)),
    )

def deceleration_zone_A_B(s):
    return Or(
        Equals(s["sector_A"], Int(1)),
        Equals(s["sector_A"], Int(3)),
        Equals(s["sector_A"], Int(12)),
        Equals(s["sector_A"], Int(14)),
    )
def v_constant(s): # e igual para as duas rotas
    return Or(
        Equals(s["sector_A"], Int(5)),
        Equals(s["sector_A"], Int(7)),
        Equals(s["sector_A"], Int(9)),
        Equals(s["sector_A"], Int(6)),
        Equals(s["sector_A"], Int(8)),
        Equals(s["sector_A"], Int(10))
    )
def low_speed(s): # e igual para as duas rotas
    return Equals(s["sector_A"],Int(0))

def mode_A_B(s):
    return Or(
        acceleration_zone_A_B(s),
        deceleration_zone_A_B(s),
        v_constant(s),
        low_speed(s)
    )
"""
acceleration_zone_A_B(s)  # γ ≈ 1 Implies(acceleration_zone_A_B(s), Equals(s["gamma_A"], Real(1)))
deceleration_zone_A_B(s)  # γ ≈ 0 Implies(deceleration_zone_A_B(s), Equals(s["gamma_A"], Real(0)))
v_constant_A_B(s)         # γ ≈ constante
low_speed_A_B(s)           # γ ≈ baixo
"""
#aqui tou a colocar valores penso q seja isso
def assign_parameters_A_B(s,acceleration_zone,deceleration_zone,v_constant_zone,low_speed_zone):
    # gamma
    flow_gamma = And(
        Implies(acceleration_zone(s), Equals(s["gamma_A"], Real(1))),
        Implies(deceleration_zone(s), Equals(s["gamma_A"], Real(0))),
        Implies(v_constant_zone(s), Equals(s["gamma_A"], s["epsilon_A"])),
        Implies(low_speed_zone(s), Equals(s["gamma_A"], Real(0.1)))
    )

    # phi
    flow_phi = And(
        Implies(acceleration_zone(s), Equals(s["phi_A"], Real(0))),
        Implies(deceleration_zone(s), Equals(s["phi_A"], Real(0))),
        Implies(v_constant_zone(s), Equals(s["phi_A"], Real(0))),
        Implies(low_speed_zone(s), Equals(s["phi_A"], Real(0)))
    )

    # V (limite de velocidade)
    flow_V = And(
        Implies(acceleration_zone(s), Equals(s["V_A"], Real(1.5))),
        Implies(deceleration_zone(s), Equals(s["V_A"], Real(1.0))),
        Implies(v_constant_zone(s), Equals(s["V_A"], Real(1.2))),
        Implies(low_speed_zone(s), Equals(s["V_A"], Real(0.5)))
    )
    return And(flow_gamma, flow_phi, flow_V)

In [ ]:
# formulaas do enunciado do movimento linear
def vars_flow_A_B(s):
    flow = And(# v˙+σv=γ se v≤V => v˙=γ−σv
        Implies(LT(s["v_A"], s["V_A"]), Equals(s["v_A_dot"],Minus( s["gamma_A"] , Times(s["sigma_A"],s["v_A"])))),
        #v˙+σv=ε se v>V => v˙=ε−σv
        Implies(GT(s["v_A"], s["V_A"]), Equals(s["v_A_dot"],Minus (s["epsilon_A"] , Times(s["sigma_A"],s["v_A"])))),

        #  # z˙ = v
        Equals(s["z_A_dot"], s["v_A"]),
        # τ˙ = 1

        Equals(s["tau_A_dot"], Real(1)),
        #ver se e aqui
        """
Se phi for constante por sector (como no teu enunciado, φ = 0):
Então cos(phi_A) = 1 e sin(phi_A) = 0
        """
        Equals(s["x_A"], Plus(s["x0_A"], s["z_A"])),  # horizontal
        Equals(s["y_A"], s["y0_A"])                  # vertical constante
        And(
            GE(Plus(s["x0_A"], s["z_A"]), Real(0)),
            LE(Plus(s["x0_A"], s["z_A"]), Real(1)),
            GE(s["y0_A"], Real(0)),
            LE(s["y0_A"], Real(1))
        )
    )
    return flow

In [7]:
def jump_next_sector_A(s, next_sector, x0_next, y0_next, semaforo_green):
    guard = And(GE(s["z_A"], Real(1)), can_advance_A(s))


    reset = And(
        Equals(s["z_A"], Real(0)),
        Equals(s["tau_A"], Real(0)),
        Equals(s["x0_A"], Real(x0_next)),
        Equals(s["y0_A"], Real(y0_next)),
        Equals(s["sector_A"], Int(next_sector))
    )

    return And(guard, reset)

In [8]:
def VAR_B_A(i):
    s = {}
    # estado discreto
    s["sector_B"] = Symbol("sector_A" + str(i), INT)

    # variáveis contínuas de estado
    s["z_B"]   = Symbol("z_B"+ str(i), REAL)
    s["v_B"]   = Symbol("v_B"+ str(i), REAL)
    s["tau_B"] = Symbol("tau_B"+ str(i), REAL)

    #sendo $$(x_0,y_0)\,$$ o ponto do sector onde a trajetória se inicia, no inicio inicio é (0,1)
    s["x0_B"] = Symbol("x0_B"+ str(i), REAL)
    s["y0_B"] = Symbol("y0_B"+ str(i), REAL)

    # parâmetros associados ao modo (sector)
    # Os modos do navio  são definidos pelos sectores $$\,s\,$$. 
    # A cada modo estão associados  parâmetros racionais, constantes dentro de cada sector mas variando de sector para sector.

    s["gamma_B"]   = Symbol("gamma_B"+ str(i), REAL) #  aceleração linear  definida pela propulsão do navio
    s["epsilon_B"] = Symbol("epsilon_B"+ str(i), REAL) #é a  força exercida pela corrente no canal que actua a favor ou contra a trajectória
    s["phi_B"]     = Symbol("phi_B"+ str(i), REAL) #  é o rumo do navio definido como o ângulo que a trajetória do movimento linear faz com o eixo horizontal.
    s["V_B"]       = Symbol("V_B"+ str(i), REAL) # limite superior na velocidade do navio

    # o coeficiente de atrito com a àgua, uma constante positiva que é independente do modo mas depende do navio.
    s["sigma_B"] = Symbol("sigma_B"+ str(i), REAL)

    s["v_B_dot"] = Symbol("v_B_dot", REAL)
    s["z_B_dot"] = Symbol("z_B_dot", REAL)
    return s

In [9]:
# nao sei  se e para colocar isto (:)
def init_B_A(s):
    return And(
        Equals(s["sector_A"], Int(12)), 
        Equals(s["z_A"], Real(0)),
        GE(s["v_A"], Real(0)),
        Equals(s["tau_A"],Real(0)),
        Equals(s["x0_A"], Real(7)),
        Equals(s["y0_A"], Real(1)),
    )


In [10]:
# os modos de aceleracao e assim
def acceleration_zone_B_A(s):
    return Or(
        Equals(s["sector_A"], Int(1)),
        Equals(s["sector_A"], Int(3)),
        Equals(s["sector_A"], Int(12)),
        Equals(s["sector_A"], Int(14)),
    )

def deceleration_zone_B_A(s):
    return Or(
        Equals(s["sector_A"], Int(11)),
        Equals(s["sector_A"], Int(13)),
        Equals(s["sector_A"], Int(2)),
        Equals(s["sector_A"], Int(4)),
    )

def mode_B_A(s):
    return Or(
        acceleration_zone_B_A(s),
        deceleration_zone_B_A(s),
        v_constant(s),
        low_speed(s)
    )

In [11]:
def assign_parameters_B_A(s, acceleration_zone, deceleration_zone, v_constant_zone, low_speed_zone):
    # gamma
    flow_gamma = And(
        Implies(acceleration_zone(s), Equals(s["gamma_B"], Real(1))),
        Implies(deceleration_zone(s), Equals(s["gamma_B"], Real(0))),
        Implies(v_constant_zone(s), Equals(s["gamma_B"], s["epsilon_B"])),
        Implies(low_speed_zone(s), Equals(s["gamma_B"], Real(0.1)))
    )

    # phi
    flow_phi = And(
        Implies(acceleration_zone(s), Equals(s["phi_B"], Real(0))),
        Implies(deceleration_zone(s), Equals(s["phi_B"], Real(0))),
        Implies(v_constant_zone(s), Equals(s["phi_B"], Real(0))),
        Implies(low_speed_zone(s), Equals(s["phi_B"], Real(0)))
    )

    # V (limite de velocidade)
    flow_V = And(
        Implies(acceleration_zone(s), Equals(s["V_B"], Real(1.5))),
        Implies(deceleration_zone(s), Equals(s["V_B"], Real(1.0))),
        Implies(v_constant_zone(s), Equals(s["V_B"], Real(1.2))),
        Implies(low_speed_zone(s), Equals(s["V_B"], Real(0.5)))
    )

    return And(flow_gamma, flow_phi, flow_V)


In [ ]:
def vars_flow_B_A(s):
    flow = And(
        Implies(LT(s["v_B"], s["V_B"]),
                Equals(s["v_B_dot"], Minus(s["gamma_B"], Times(s["sigma_B"], s["v_B"])))),
        Implies(GT(s["v_B"], s["V_B"]),
                Equals(s["v_B_dot"], Minus(s["epsilon_B"], Times(s["sigma_B"], s["v_B"])))),
        
        Equals(s["z_B_dot"], s["v_B"]),

        Equals(s["tau_B_dot"], Real(1)),
#cos(ϕB​)=1, sin(ϕB​)=0
        Equals(s["x_B"], Plus(s["x0_B"], s["z_B"])),
        Equals(s["y_B"], s["y0_B"]),

        And(
            GE(Plus(s["x0_B"], s["z_B"]), Real(0)),
            LE(Plus(s["x0_B"], s["z_B"]), Real(1)),
            GE(s["y0_B"], Real(0)),
            LE(s["y0_B"], Real(1))
        )
    )
    return flow

as formulas de movimento linear é igual


In [12]:
def jump_next_sector_B(s, next_sector, x0_next, y0_next, semaforo_green):
    guard = And(GE(s["z_B"], Real(1)), can_advance_B(s))


    reset = And(
        Equals(s["z_B"], Real(0)),
        Equals(s["tau_B"], Real(0)),
        Equals(s["x0_B"], Real(x0_next)),
        Equals(s["y0_B"], Real(y0_next)),
        Equals(s["sector_B"], Int(next_sector))
    )

    return And(guard, reset)